<a href="https://colab.research.google.com/github/Sam-wiz/cocacola-pepsi-detection-yolo/blob/master/pepsi_coca_cola_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/

/content/drive/MyDrive


In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 16799, done.
remote: Counting objects: 100% (340/340), done.
remote: Compressing objects: 100% (245/245), done.
remote: Total 16799 (delta 185), reused 180 (delta 95), pack-reused 16459
Receiving objects: 100% (16799/16799), 15.58 MiB | 10.78 MiB/s, done.
Resolving deltas: 100% (11488/11488), done.
Updating files: 100% (148/148), done.


In [ ]:
%cd yolov5

/content/drive/MyDrive/yolov5


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.7/802.7 kB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (

In [ ]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
import zipfile
import os

zip_ref = zipfile.ZipFile('dataset.zip', 'r')
zip_ref.extractall('/content/drive/MyDrive/yolov5/dataset')
zip_ref.close()

In [ ]:
os.listdir('/content/drive/MyDrive/yolov5/dataset')

['dataset', '__MACOSX']

In [ ]:
%cd yolov5

/content/drive/MyDrive/yolov5


In [ ]:
%ls

benchmarks.py    data/      hubconf.py  __pycache__/     requirements.txt  utils/
CITATION.cff     dataset/   inp.mp4     pyproject.toml   segment/          val.py
classify/        detect.py  LICENSE     README.md        train.py
CONTRIBUTING.md  export.py  models/     README.zh-CN.md  tutorial.ipynb


In [11]:
!python train.py --img 640 --batch 16 --epochs 50 --data data.yaml --weights yolov5s.pt

2024-07-17 06:31:41.181721: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-17 06:31:41.181777: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-17 06:31:41.183893: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optim

In [ ]:
!pip install av opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 30.5 MB/s eta 0:00:00


In [2]:
%cd drive/MyDrive/yolov5/

/content/drive/MyDrive/yolov5


In [3]:
import av
import cv2
import torch
import json
from collections import defaultdict
import numpy as np

# Load YOLO model
model_path = 'runs/train/exp/weights/best.pt'  # Replace with your trained model path
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)

# Function to extract frames and their timestamps in batches
def extract_frames(video_path, batch_size=32):
    container = av.open(video_path)
    frames = []
    timestamps = []
    batch_count = 0

    for frame in container.decode(video=0):
        img = frame.to_image()
        img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        frames.append(img)
        timestamps.append(frame.time)

        if len(frames) == batch_size:
            yield frames, timestamps
            frames = []
            timestamps = []
            batch_count += 1
            print(f"Processed batch {batch_count}")

    if frames:
        yield frames, timestamps

# Function to detect logos and collect timestamps
def detect_logos(frames, timestamps):
    results = defaultdict(list)
    for frame, timestamp in zip(frames, timestamps):
        results_list = model(frame).pandas().xyxy[0].to_dict(orient="records")
        for result in results_list:
            if result['name'] == 'Coca-cola':
                results['CocaCola_pts'].append(timestamp)
            elif result['name'] == 'Pepsi':
                results['Pepsi_pts'].append(timestamp)
    return results

# Main function to process the video and output JSON
def process_video(video_path, output_json, batch_size=32):
    final_results = defaultdict(list)

    for frames, timestamps in extract_frames(video_path, batch_size):
        batch_results = detect_logos(frames, timestamps)
        for key, value in batch_results.items():
            final_results[key].extend(value)

    with open(output_json, 'w') as f:
        json.dump(final_results, f, indent=4)

    print(f"Results saved to {output_json}")

# Example usage
video_path = 'inp.mp4'  # Replace with the path to your video file
output_json = 'output.json'
process_video(video_path, output_json, batch_size=32)


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-7-17 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
WARNING ⚠️ NMS time limit 0.550s exceeded


Processed batch 1
Processed batch 2
Processed batch 3
Processed batch 4
Processed batch 5
Processed batch 6
Processed batch 7
Processed batch 8
Processed batch 9
Processed batch 10
Processed batch 11
Processed batch 12
Processed batch 13
Processed batch 14
Processed batch 15
Processed batch 16
Processed batch 17
Processed batch 18
Processed batch 19
Processed batch 20
Processed batch 21
Processed batch 22
Processed batch 23
Processed batch 24
Processed batch 25
Processed batch 26
Processed batch 27
Processed batch 28
Processed batch 29
Processed batch 30
Processed batch 31
Processed batch 32
Processed batch 33
Processed batch 34
Processed batch 35
Processed batch 36
Processed batch 37
Processed batch 38
Processed batch 39
Processed batch 40
Processed batch 41
Processed batch 42
Processed batch 43
Processed batch 44
Processed batch 45
Processed batch 46
Processed batch 47
Processed batch 48
Processed batch 49
Processed batch 50
Processed batch 51
Processed batch 52
Processed batch 53
Pr